In [ ]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time, random
import numpy as np, pandas as pd, seaborn as sns
from tqdm import tqdm
from typing import Dict

#for deep learning
import torch
from torch import nn, optim, Tensor
from torchvision.models.resnet import resnet18

#for scene visualization
from IPython.display import display, clear_output
from IPython.display import HTML
import PIL
from matplotlib import animation, rc

os.chdir('/kaggle/input')

# Overview

**Most of the contents of this notebook can be found on the `l5kit` GitHub repository in the `examples` folder [here](https://github.com/lyft/l5kit/blob/master/examples). It is a great place to start for this competition.**

# Data Format

**The data in this competition is packed into `.zarr` files, which can be loaded with the Python `zarr` module. Taken from the competition Data description, each `.zarr` file contains a set of:**

* scenes: driving episodes acquired from a given vehicle.
* frames: snapshots in time of the pose of the vehicle.
* agents: a generic entity captured by the vehicle's sensors. Note that only 4 of the 17 possible agent label_probabilities are present in this dataset.
* agents_mask: a mask that (for train and validation) masks out objects that aren't useful for training. In test, the mask (provided in files as mask.npz) masks out any test object for which predictions are NOT required.
* traffic_light_faces: traffic light information.

# l5kit

**`l5kit` is a Python library developed by Lyft Level 5 with functionality for developing and training *learned prediction, planning and simulation* models for autonomous driving applications. Among other things, this library allows us to predict the future movement of cars based on historical observations. In particular, `l5kit` allows us to:**

* Load driving scenes from zarr files
* Read aerial and semantic maps
* Create birds-eye-view images that repesent a scene around an automonous vehicle/other vehicles
* And most importantly, train neural networks and visualize their results

**Let's add `l5kit` now and start to explore how we can use it for the task at hand. Note that as per the competition rules, internet is *not* allowed in this competition, so we cannot use `pip` to install packages. This is why Lyft has provided us the `l5kit` module as a utility script called `kaggle_l5kit`.**

## Data

**Data is expected to live in the folder that we set using the `L5KIT_DATA_FOLD` env variable. This folder should contain subfolders for the aerial and semantic maps as well as the `.zarr` files (the scenes). Luckily, this is exactly how our data is set up in this Kaggle kernel so all we need to do is this:** 


In [ ]:
# set env variable for data
PATH_TO_DATA = "../input/lyft-motion-prediction-autonomous-vehicles"
os.environ["L5KIT_DATA_FOLDER"] = PATH_TO_DATA

**Now, we can iteract with `l5kit` via the below configuration. It will become clear how to manipulate this `cfg` as we go through the notebook:**

In [ ]:
cfg = {
    'model_params': {
        'model_architecture': 'resnet18',
        
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },
    
    'sample_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 4
    },
    
    'train_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 25,
        'image_coords': True
        
    },
        
    'test_params': {
        'image_coords': True
        
    }
}

**This configuration file is used to load the data by using a `LocalDataManager` object, which resolves relative paths from the configuration file using the `L5KIT_DATA_FOLDER` env that we set earlier, to extract the `.zarr` file and store it as a `ChunkedDataset` object. When looking at the above configuration dictionary, we see that the `.zarr` files of interest are in `train_data_loader : key`:**

In [ ]:
from l5kit.data import LocalDataManager, ChunkedDataset

dm = LocalDataManager()
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()

#let's see what one of the objects looks like
print(train_zarr)

# Visualization

**Main source: https://github.com/lyft/l5kit/tree/master/examples/visualisation**

### Raw Data

**`.zarr` files support most traditional NumPy array operations. For example, we can iterate over the frames in our `.zarr` file to create a scatter plot of all the AV (henceforth AV) locations like so:**

**I have this #-d out because it takes around 40 minutes to complete and is not nearly as insightful as the below visualizations will be:**

In [ ]:
#frames = train_zarr.frames
#coords = np.zeros((len(frames), 2))
#for idx_coord, idx_data in enumerate(tqdm(range(len(frames)), desc="getting centroid to plot trajectory")):
    #frame = train_zarr.frames[idx_data]
    #coords[idx_coord] = frame["ego_translation"][:2]


#plt.scatter(coords[:, 0], coords[:, 1], marker='.')
#axes = plt.gca()
#axes.set_xlim([-2500, 1600])
#axes.set_ylim([-2500, 1600])

### Data Abstraction

**Working with raw `.zarr` files is fine, but `l5kit` also provides abstract data classes to generate inputs and targets with ease. Most importantly, we can use a `rasterizer`, which takes these chunked `.zarr` files and processes them into rectangular grid of pixels (a raster image) so that we can view them regularly on a computer screen.**

In [ ]:
from l5kit.rasterization import build_rasterizer

rasterizer = build_rasterizer(cfg, dm)

**We can then use the `EgoDataset` and `AgentDataset` objects in `l5kit` to iterate over the rasterizer object we just defined to return RGB images. The `EgoDataset` iterates only over the AV annotations and the `AgentDataset` iterates everything but the AV annotations. Let's first visualize the AV using an `EgoDataset`:**

In [ ]:
from l5kit.dataset import EgoDataset

AV_ds = EgoDataset(cfg, train_zarr, rasterizer)

**Once in an `EgoDataset`, we can plot the ground truth trajectory by converting the `EgoDataset.target_position` into pixel coordinates and then call `draw_trajectory`, which can be used to draw predicted trajectories as well.**

In [ ]:
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points

sample = AV_ds[50]

im = sample["image"].transpose(1, 2, 0)
im = AV_ds.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(sample["target_positions"] + sample["centroid"][:2], sample["world_to_image"])
draw_trajectory(im, target_positions_pixels, sample["target_yaws"], TARGET_POINTS_COLOR)

_, ax = plt.subplots(figsize = (7, 7))
plt.imshow(im[::-1])
plt.show()

**We can change the `rasterizer` by building a new one and an accomodating dataset for it like so:**

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rasterizer = build_rasterizer(cfg, dm)
AV_ds = EgoDataset(cfg, train_zarr, rasterizer)
sample = AV_ds[50]

im = sample["image"].transpose(1, 2, 0)
im = AV_ds.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(sample["target_positions"] + sample["centroid"][:2], sample["world_to_image"])
draw_trajectory(im, target_positions_pixels, sample["target_yaws"], TARGET_POINTS_COLOR)

_, ax = plt.subplots(figsize = (7, 7))
plt.imshow(im[::-1])
plt.show()

**And now we can do the exact same thing to view the agents by switching from an `EgoDataset` to an `AgentDataset`:**

In [ ]:
from l5kit.dataset import AgentDataset

cfg["raster_params"]["map_type"] = "py_semantic"
A_ds = AgentDataset(cfg, train_zarr, rasterizer)
sample = A_ds[50]

im = sample["image"].transpose(1, 2, 0)
im = A_ds.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(sample["target_positions"] + sample["centroid"][:2], sample["world_to_image"])
draw_trajectory(im, target_positions_pixels, sample["target_yaws"], TARGET_POINTS_COLOR)

_, ax = plt.subplots(figsize = (7, 7))
plt.imshow(im[::-1])
plt.show()

**The below animation code is taken from [here](https://www.kaggle.com/nxrprime/lyft-understanding-the-data-and-eda):**

In [ ]:
def animate_solution(images):
    def animate(i):
        im.set_data(images[i])
        
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames = len(images), interval = 60)
 
cfg["raster_params"]["map_type"] = "py_semantic"
rasterizer = build_rasterizer(cfg, dm)
A_ds = EgoDataset(cfg, train_zarr, rasterizer)
scene_idx = 34
indexes = AV_ds.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    data = A_ds[idx]
    im = data["image"].transpose(1, 2, 0)
    im = A_ds.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
    
HTML(animate_solution(images).to_jshtml())

**Pretty cool right? We are able to follow our AV as it traverses threw this scene**

# PyTorch Training

**Main source: https://github.com/lyft/l5kit/tree/master/examples/agent_motion_prediction**

**And now we are ready to explore how to train a neural network with this `.zarr` data. To prepare a rar `.zarr` file for training, we follow the below procedure:**

1. Load `.zarr` file into a `ChunkedDataset` object
2. Wrap `ChunkedDataset` object into an `AgentDataset`, which inhereits from the PyTorch `Dataset` class
3. Pass `AgentDataset` into a torch `Dataloader`

**Let's do this now:**

In [ ]:
from torch.utils.data import DataLoader

train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset,
                              shuffle = cfg["train_data_loader"]["shuffle"],
                              batch_size = cfg["train_data_loader"]["batch_size"],
                              num_workers = cfg["train_data_loader"]["num_workers"])

print(len(train_dataloader))

### Model

**We can create a simple model to feed this `DataLoader` to. The below is a ResNet18 pre-trained on ImagNet taken from [Peter](https://www.kaggle.com/pestipeti)'s notebook [here](https://www.kaggle.com/pestipeti/pytorch-baseline-train). We need to change the input channels of the ResNet to match the `rasterizer` output. We also need to change the output size to `(X, Y) * number of future states`:**

**Update: see discussion post [here](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/186492) for explanation of new architecture and training loop changes:**

In [ ]:
class LyftModel(nn.Module):

    def __init__(self, cfg: Dict, num_modes=3):
        super().__init__()

        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True, progress=True)
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512

        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.logit = nn.Linear(4096, out_features=self.num_preds + num_modes)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)
        x = self.logit(x)
        
        return x

In [ ]:
def forward(data, model, device, criterion):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    matrix = data["world_to_image"].to(device)
    centroid = data["centroid"].to(device)[:,None,:].to(torch.float)

    # Forward pass
    outputs = model(inputs)

    bs,tl,_ = targets.shape
    assert tl == cfg["model_params"]["future_num_frames"]

    if cfg['train_params']['image_coords']:
        targets = targets + centroid
        targets = torch.cat([targets,torch.ones((bs,tl,1)).to(device)], dim=2)
        targets = torch.matmul(matrix.to(torch.float), targets.transpose(1,2))
        targets = targets.transpose(1,2)[:,:,:2]
        rs = cfg["raster_params"]["raster_size"]
        ec = cfg["raster_params"]["ego_center"]

        bias = torch.tensor([rs[0] * ec[0], rs[1] * ec[1]])[None, None, :].to(device)
        targets = targets - bias

    confidences, pred = outputs[:,:3], outputs[:,3:]
    pred = pred.view(bs, 3, tl, 2)
    assert confidences.shape == (bs, 3)
    confidences = torch.softmax(confidences, dim=1)

    loss = criterion(targets, pred, confidences, target_availabilities)
    loss = torch.mean(loss)

    if cfg['train_params']['image_coords']:
        matrix_inv = torch.inverse(matrix)
        pred = pred + bias[:,None,:,:]
        pred = torch.cat([pred,torch.ones((bs,3,tl,1)).to(device)], dim=3)
        pred = torch.stack([torch.matmul(matrix_inv.to(torch.float), pred[:,i].transpose(1,2)) 
                            for i in range(3)], dim=1)
        pred = pred.transpose(2,3)[:,:,:,:2]
        pred = pred - centroid[:,None,:,:]

    return loss, pred, confidences

In [ ]:
# from https://www.kaggle.com/corochann/lyft-training-with-multi-mode-confidence
def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:

    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability
    
    if cfg['train_params']['image_coords']:
        error = error / 4

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:

    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

**We can speed up our training by placing our computations on the GPU with the `torch.device` object. We will define our optimizer and metric here as well:**

### Training Loop

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftModel(cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
criterion = pytorch_neg_multi_log_likelihood_batch

In [ ]:
if device.type == 'cpu': print('Training on CPU')
if device.type == 'cuda': print('Training on GPU')

In [ ]:
tr_it = iter(train_dataloader)

progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []

for itr in progress_bar:

    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)

    model.train()
    torch.set_grad_enabled(True)

    loss, pred, confidences = forward(data, model, device, criterion)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
        
    #save model during training
    if (itr+1) % cfg['train_params']['checkpoint_every_n_steps'] == 0 and not DEBUG:
        torch.save(model.state_dict(), f'model_state_{itr}.pth')
    
    #display training progress
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

**Once you are happy with your training, you can save the model state to your disk and upload the pre-trained weights offline for an inference submission using `torch.save`**

# Inference

**And now, after all that work, we can predict with this baseline model. The following code is (again) taken from [Peter](https://www.kaggle.com/pestipeti)'s notebook [here](https://www.kaggle.com/pestipeti/pytorch-baseline-inference):**

**Since I have internet enabled in this notebook (which is not allowed for submission notebooks in this comp), I carried out the prediction part in [this notebook](https://www.kaggle.com/tuckerarrants/lyft-inference-resnet50/edit), but I included the general procedure here for completeness**

In [ ]:
test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{PATH_TO_DATA}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataset)

In [ ]:
def run_prediction(predictor, data_loader):
    predictor.eval()

    pred_coords_list = []
    confidences_list = []
    timestamps_list = []
    track_id_list = []

    with torch.no_grad():
        dataiter = tqdm(data_loader)
        for data in dataiter:
            image = data["image"].to(device)
            inputs = data["image"].to(device)
            target_availabilities = data["target_availabilities"].to(device)
            targets = data["target_positions"].to(device)
            matrix = data["world_to_image"].to(device)
            centroid = data["centroid"].to(device)[:,None,:].to(torch.float)
            
            pred, confidences = predictor(image)
            
            if cfg['test_params']['image_coords']:
                matrix_inv = torch.inverse(matrix)
                pred = pred + bias[:,None,:,:]
                pred = torch.cat([pred,torch.ones((bs,3,tl,1)).to(device)], dim=3)
                pred = torch.stack([torch.matmul(matrix_inv.to(torch.float), pred[:,i].transpose(1,2)) 
                                    for i in range(3)], dim=1)
                pred = pred.transpose(2,3)[:,:,:,:2]
                pred = pred - centroid[:,None,:,:]

            pred_coords_list.append(pred.cpu().numpy().copy())
            confidences_list.append(confidences.cpu().numpy().copy())
            timestamps_list.append(data["timestamp"].numpy().copy())
            track_id_list.append(data["track_id"].numpy().copy())
            
    timestamps = np.concatenate(timestamps_list)
    track_ids = np.concatenate(track_id_list)
    coords = np.concatenate(pred_coords_list)
    confs = np.concatenate(confidences_list)
    return timestamps, track_ids, coords, confs

**Here you upload your freshly trained model:**

In [ ]:
LOAD_MODEL = False

if LOAD_MODEL:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    predictor = LyftModel(cfg)
    saved_model_path = ""
    predictor.load_state_dict(torch.load(saved_model_path))
    predictor.to(device)

**Now we get our predictions and save them as a `csv` file with `write_pred_csv`:**

In [ ]:
from l5kit.evaluation import write_pred_csv

INFER = False

if INFER:
    timestamps, track_ids, coords, confs = run_prediction(predictor, test_dataloader)
    write_pred_csv('submission.csv',timestamps=timestamps,
    track_ids=track_ids, coords=coords, confs=confs)

# Training Parameter EDA

**Let's see which parameters in our configuration file are relevant to training:**

In [ ]:
train_dataset[0].keys()

**Let's explore two important training specific parameters: `raster_size` and `pixel_size`:**

### Raster Size

In [ ]:
def sample_dataset():
    # Build Rasterizer
    rasterizer = build_rasterizer(cfg, dm)
    
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
    return train_dataset[100]

In [ ]:
sizes = [[200, 200], [224, 224], [250, 250], [350, 350], [450, 450], [500, 500]]

f, ax = plt.subplots(2, 3, figsize=(20, 12))
ax = ax.flatten()

for i in range(6):
    cfg['raster_params']['raster_size'] = sizes[i]
    sample = sample_dataset()
    
    ax[i].imshow(sample['image'][-3:].transpose(1, 2, 0))
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(f"Raster size: {sizes[i]}")

#reset to default
cfg['raster_params']['raster_size'] = [224, 224]

**We can see that different raster sizes affect how much of a scene is rasterized, meaning that our model sees different agents depending on the raster size. This is a powerful hyper parameter: should our model focus on agents far away? Or should it focus on agents closer to it?**

### Pixel Size

**From the `l5kit` GitHub repository, the pixel size of the raster image is the:**

> raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.

**So this is the value to tweak if you want to change the resolution of the image. This is yet another import training parameter to experiment with, especially when we are feeding these rasterized images to a CNN**

In [ ]:
sizes = [[.2, .2,], [.3, .3], [.4, .4], [.5, .5], [.6, .6], [.7, .7]]

f, ax = plt.subplots(2, 3, figsize=(20, 12))
ax = ax.flatten()

for i in range(6):
    cfg['raster_params']['pixel_size'] = sizes[i]
    sample = sample_dataset()
    
    ax[i].imshow(sample['image'][-3:].transpose(1, 2, 0))
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(f"Pixel size: {sizes[i]}")

#reset to default
cfg['raster_params']['pixel_size'] = [0.5, 0.5]

**If `pixel_size = [.5, .5]`, this means that we are at most able to capture objects no less than half a meter in size: if the object is smaller than half a meter, it will not be detected as a pixel, so to view these objects, we need a higher resolution (lower `pixel_size`)**

**As you can read about in this [notebook](https://www.kaggle.com/ilu000/expected-error-inaccuracy-from-rasterization), you'll see that there is some inherent innacuracy from the rasterization process defined by `pixel_size`. Each history position, lane, and other agents are encoded into the pixels and if the raster has `pixel_size = .5`, we ought to expect a mean error of `.5/4` for each direction of each predicted position. We can combat this by using a smaller `pixel_size`:**

### Changing Raster Size and Pixel Size

In [ ]:
sizes = [[[200, 200],[.5, .5]], [[250, 250],[.4, .4]], [[350, 350],[.3, .3]], [[500, 500],[.2, .2]]]

f, ax = plt.subplots(1, 4, figsize=(20, 12))
ax = ax.flatten()

for i in range(4):
    cfg['raster_params']['pixel_size'] = sizes[i][1]
    cfg['raster_params']['raster_size'] = sizes[i][0]
    sample = sample_dataset()
    
    ax[i].imshow(sample['image'][-3:].transpose(1, 2, 0))
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(f"Raster/Pixel size: {sizes[i]}")

#reset to default
cfg['raster_params']['raster_size'] = [224, 224]

**If you want the model to see more of a scene, you can change either `raster_size` or `pixel_size`. More importantly, if you want the same scene in a higher resolution, you need to change *both* `raster_size` and `pixel_size`. For a more thorough explanation, see Peter's comment [here](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/178323)**

In [ ]:
sizes = [[[400, 400],[.2, .2]], [[500, 500],[.2, .2]], [[600, 600],[.2, .2]], [[700, 700],[.2, .2]]]

f, ax = plt.subplots(1, 4, figsize=(20, 12))
ax = ax.flatten()

for i in range(4):
    cfg['raster_params']['pixel_size'] = sizes[i][1]
    cfg['raster_params']['raster_size'] = sizes[i][0]
    sample = sample_dataset()
    
    ax[i].imshow(sample['image'][-3:].transpose(1, 2, 0))
    ax[i].get_xaxis().set_visible(False)
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title(f"Raster/Pixel size: {sizes[i]}")

# Augmentation

**It seems there is a large gap between training loss and validation. This is largely because some images are very similar, e.g. the frames of the agent vehicle at a red light are much the same in comparison to frames in which it is moving at full speed.**

**We can reduce this similarity between frames by introducing augmentations like CutOut, CourseDropout, and more. `l5kit` uses PyTorch `DataLoaders` so we will use `albumentations` to compose these augmentations.**

In [ ]:
#for augmentations
import albumentations as A

#for better visualization
cfg['raster_params']['raster_size'] = [500, 500]
cfg['raster_params']['pixle_size'] = [.25, .25]

In [ ]:
def show_images(aug_cfg):
    dm = LocalDataManager()
    train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
    cfg["raster_params"]["map_type"] = 'py_semantic'
    rasterizer = build_rasterizer(cfg, dm)
    sem_ds = AgentDataset(cfg, train_zarr, rasterizer)
    cfg["raster_params"]["map_type"] = 'py_satellite'
    rasterizer = build_rasterizer(cfg, dm)
    sat_ds = AgentDataset(cfg, train_zarr, rasterizer)

    #get a random sample
    random_index = int(np.random.random()*len(AV_ds))
    sat_sample = sat_ds[random_index]
    sem_sample = sem_ds[random_index]

    sat_im = sat_sample["image"].transpose(1, 2, 0)
    sat_im = sat_ds.rasterizer.to_rgb(sat_im)
    sem_im = sem_sample["image"].transpose(1, 2, 0)
    sem_im = sem_ds.rasterizer.to_rgb(sem_im)
    
    fig, ax = plt.subplots(len(aug_cfg), 2, figsize=(15,15))
    
    for i, (key, aug) in enumerate(aug_cfg.items()):
        if aug is None:
            ax[i, 0].imshow(sat_im[::-1])
            ax[i, 0].set_title(key)
        else:
            sat_im_ = aug(image=sat_im)['image']
            ax[i, 0].imshow(sat_im_[::-1])
            ax[i, 0].set_title(key)
            
    for i, (key, aug) in enumerate(aug_cfg.items()):
        if aug is None:
            ax[i, 1].imshow(sem_im[::-1])
            ax[i, 1].set_title(key)
        else:
            sem_im_ = aug(image=sem_im)['image']
            ax[i, 1].imshow(sem_im_[::-1])
            ax[i, 1].set_title(key)

    plt.tight_layout();

In [ ]:
aug_dict = {'Original': None,
            
            'Cutout':A.Cutout(num_holes=10, max_h_size=20, max_w_size=20, fill_value=0, 
                              always_apply=False, p=1),
            
            'CoarseDropout': A.CoarseDropout(max_holes=10, max_height=20, max_width=20, 
                                             min_holes=None, min_height=None, min_width=None,
                                             fill_value=0, always_apply=False, p=1),
            
            'GridDropout': A.GridDropout(ratio=.4, unit_size_min=None, unit_size_max=None,
                                         holes_number_x=None, holes_number_y=None,
                                         shift_x=0, shift_y=0, p=1)}

show_images(aug_dict)

**Now this probably isn't that useful, but it is cool so I will demo it anyway. We can add some  weather augmentations, which doesn't make much sense to do with the `py_semantic` raster images, but I guess one could use it for `py_satellite` raster images.**

In [ ]:
aug_dict = {'Original': None,
            
            'RandomRain': A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20,
                                      drop_width=1,blur_value=7, brightness_coefficient=0.7,
                                      rain_type=None, always_apply=False, p=1),
            
            'RandomFog': A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=1, alpha_coef=0.08,
                                     always_apply=False, p=1),
            
            'RandomSnow': A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.3,
                                        brightness_coeff=2.5, always_apply=False, p=1)}

show_images(aug_dict)

# To be continued...